In [ ]:
# Necessary imports
import os
import pathlib
os.environ['USE_PYGEOS'] = '0'
import gc
import numpy as np
from itertools import product
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors
import ee

import pandas as pd
from georeader.readers import ee_query
import folium
import geemap.foliumap as geemap
import shapely
from shapely.geometry import box, Point, MultiPoint
from shapely.ops import nearest_points

from skimage.morphology import binary_dilation, disk

from typing import Tuple, Callable, Any, Optional


from ml4floods.data.worldfloods import dataset
from ml4floods.data import utils
from ml4floods.visualization import plot_utils
from ml4floods.models import postprocess

from db_utils import DB

# Uncomment this to suppress deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

# Set bucket will not be requester pays
utils.REQUESTER_PAYS_DEFAULT = False

In [ ]:
import rasterio
from rasterio.io import MemoryFile
from rasterio import Affine as A
from rasterio import CRS
#from rasterio.warp import reproject, Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.transform import from_origin


## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"

print("[INFO] Successfully loaded FloodMapper environment.")

In [ ]:
# Connect to the database (point to the .env file for credentials)
db_conn = DB(env_file_path)

In [ ]:
# Initialise the Google Earth Engine connection.
# Follow instructions on login prompt, if required.
ee.Initialize()

In [ ]:
# EDIT THIS CELL: All work is conducted under a unique session name
session_name = "boulia_test"

## Retrieve the session parameters from the database

Edit the following cell to set the session name and retrieve the flood map parameters, including mapping grid and affected LGAs.

In [ ]:
tz = ZoneInfo("UTC")
midnight = datetime.min.time()

# Query the floodmapping parameters from the DB
query = (f"SELECT flood_date_start, flood_date_end, "
         f"ref_date_start, ref_date_end, bucket_uri "
         f"FROM session_info "
         f"WHERE session = %s;")
data = (session_name,)
session_df = db_conn.run_query(query, data, fetch=True)
flood_start_date = session_df.iloc[0]["flood_date_start"]
flood_start_date = datetime.combine(flood_start_date, midnight).replace(tzinfo=tz)
flood_end_date = session_df.iloc[0]["flood_date_end"]
flood_end_date = datetime.combine(flood_end_date, midnight).replace(tzinfo=tz)
ref_start_date = session_df.iloc[0]["ref_date_start"]
ref_start_date = datetime.combine(ref_start_date, midnight).replace(tzinfo=tz)
ref_end_date = session_df.iloc[0]["ref_date_end"]
ref_end_date = datetime.combine(ref_end_date, midnight).replace(tzinfo=tz)
bucket_uri = session_df.iloc[0]["bucket_uri"]

# Query the selected grid positions and LGAs
query = (f"SELECT sp.patch_name, ST_AsText(gr.geometry), gr.lga_name22 "
         f"FROM session_patches sp "
         f"INNER JOIN grid_loc gr "
         f"ON sp.patch_name = gr.patch_name "
         f"WHERE sp.session = %s ;")
data = (session_name,)
grid_sel_df = db_conn.run_query(query, data, fetch=True)

# Format the results into a correct GeoDataFrame
grid_sel_df['geometry'] = gpd.GeoSeries.from_wkt(grid_sel_df['st_astext'])
grid_sel_df.drop(['st_astext'], axis=1, inplace = True)
grid_sel_gdf = gpd.GeoDataFrame(grid_sel_df, geometry='geometry', crs="EPSG:4326")
grid_sel_gdf.drop_duplicates(subset=["patch_name"], inplace=True)
print(f"[INFO] {len(grid_sel_gdf)} grid patches selected.")

# Query the affected LGA shapes
lgas_sel_lst = grid_sel_df.lga_name22.unique().tolist()
query = (f"SELECT DISTINCT lga_name22, ST_AsText(geometry_col) "
         f"FROM lgas_info "
         f"WHERE lga_name22 IN %s ;")
data = (tuple(lgas_sel_lst),)
lgas_sel_df = db_conn.run_query(query, data, fetch=True)

# Format the results into a correct GeoDataFrame
lgas_sel_df['geometry'] = gpd.GeoSeries.from_wkt(lgas_sel_df['st_astext'])
lgas_sel_df.drop(['st_astext'], axis=1, inplace = True)
lgas_sel_gdf = gpd.GeoDataFrame(lgas_sel_df, geometry='geometry', crs="EPSG:4326")
print(f"[INFO] {len(lgas_sel_gdf)} LGAs affected.")

In [ ]:
# Construct the GCP paths
rel_grid_path = "0_DEV/1_Staging/GRID"
rel_operation_path = "0_DEV/1_Staging/operational"
grid_path = os.path.join(bucket_uri, rel_grid_path).replace("\\", "/")
bucket_name = bucket_uri.replace("gs://","").split("/")[0]
session_path = os.path.join(bucket_uri,
                            rel_operation_path,
                            session_name).replace("\\", "/")

fs = utils.get_filesystem(grid_path)
print(f"[INFO] Will read inference products from:\n\t{grid_path}")
print(f"[INFO] Will write mapping products to:\n\t{session_path}")

In [ ]:
# Query the database for successful maps of each mode
query = (f"SELECT patch_name, mode, data_path "
         f"FROM postproc_temporal "
         f"WHERE session = %s AND status = %s;")
data = (session_name, 1)
temporal_df = db_conn.run_query(query, data, fetch=True)
temporal_df

In [ ]:
 # Select the files for the FLOOD map
flood_df = temporal_df.loc[temporal_df["mode"] == "flood"]
geojsons_lst = [x for x in flood_df["data_path"].values]
num_files = len(geojsons_lst)
if num_files == 0:
    print(f"\t[ERR] No flooding files to merge!")
else:
    print(f"\tSelected {num_files} grid patches for flood map.")
flood_df

In [ ]:
postprocess.spatial_aggregation(geojsons_lst)

In [ ]:
floodmap = utils.read_geojson_from_gcp(geojsons_lst[0])

In [ ]:
plot_utils.plot_floodmap(floodmap) 

In [ ]:
floodmap.crs

In [ ]:
floodmap.to_crs?